In [5]:
from dataset import download_tfds_imdb_as_text
from collections import Counter
from gensim.models import Word2Vec, KeyedVectors
from nlp_utils import spacy_tokenizer_lower_lemma_remove_stop, preprocess_remove_html_non_ascii, spacy_tokenizer,spacy_tokenizer_remove_stop

import spacy
nlp = spacy.load("en_core_web_sm")

ModuleNotFoundError: No module named 'dataset'

In [ ]:
model_word2vec = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
X_train, X_test, y_train, y_test = download_tfds_imdb_as_text()
X_train_preprocessed = [preprocess_remove_html_non_ascii(doc) for doc in X_train]
X_train_tokenized = [[e for e in nlp(doc, disable=["tagger", "parser"])] for doc in X_train_preprocessed]

X_train_tokenized_v2 = [nlp(doc) for doc in X_train_preprocessed]

In [ ]:
def filter_spacy(spacy_tok):
    return not (
        spacy_tok.is_bracket or
        spacy_tok.is_quote or
        not spacy_tok.is_ascii or
        spacy_tok.is_currency or
        spacy_tok.is_digit or
        spacy_tok.is_space or
        spacy_tok.like_email or
        spacy_tok.like_num or
        spacy_tok.like_url
    )


def filter_spacy2(spacy_tok):
    return False
    

In [ ]:
X_train_tokenized_filtered = [[e.orth_ for e in filter( filter_spacy, doc)] for doc in X_train_tokenized]


In [ ]:
counter = Counter([e for s in X_train_tokenized_filtered for e in s])
imdb_vocab = set(k for k,v in counter.items() if v >1)

In [ ]:
# lower + imdb

vocab_1 = imdb_vocab.union(set( list(model_word2vec.vocab)[:300000]))

In [ ]:
len(vocab_1)

In [ ]:
"followable" in vocab_1

In [ ]:

model = Word2Vec(size=300, window=7, min_count=1, workers=4, sg=1)
model.build_vocab([list(vocab_1)])

In [ ]:
model.intersect_word2vec_format(fname='./GoogleNews-vectors-negative300.bin',
                               lockf=0,
                               binary=True)

In [ ]:
print("riemann" in model_word2vec)
print("Dishum" in model_word2vec)
print("followable" in model_word2vec)


print("riemann" in model.wv)
print("Dishum" in model.wv)
print("followable" in model.wv)

In [ ]:
model.wv["cat"][:10]

In [ ]:
model.wv["Dishum"][:10] # look like random init

In [ ]:
model.wv["followable"][:10] # look like random init

In [2]:
model.train(X_train_tokenized_filtered_v2, total_examples=len(X_train_tokenized_filtered_v2), epochs=1)

NameError: name 'model' is not defined

In [ ]:
model.wv["followable"][:10] # after train  2 epoch

In [ ]:
model.wv["cat"][:10]

In [ ]:
model.save("word2vec.model.tf.300k.1.case.v2")

In [ ]:
dir(X_train_tokenized[0][0])


In [ ]:
[s for s in X_train_tokenized_v2[0].sents]

In [ ]:
X_train_tokenized_v2 = [nlp(doc) for doc in X_train_preprocessed]

In [ ]:
sentenized_corpus = [s for instance in X_train_tokenized_v2 for s in instance.sents ]

In [ ]:
X_train_tokenized_filtered_v2 = [[e.orth_ for e in filter( filter_spacy, doc)] for doc in sentenized_corpus]

In [ ]:
X_train_tokenized_filtered_v2_no_fulstop = [list(filter(lambda x: x!=".", s)) for s in X_train_tokenized_filtered_v2]

In [ ]:
model = Word2Vec(X_train_tokenized_filtered_v2, size=240, window=5, min_count=1, workers=4, sg=1, iter=10)
model.save("word2vec.model.240.5.10.filtered.v2")

In [ ]:
X_train_tokenized_filtered_v2_no_fulstop[1]

In [ ]:
import threading
for thread in threading.enumerate():
    print(thread.name)

In [ ]:
X_train_new = [e.orth_.lower() for sent in X_train_tokenized for e in sent]
X_train, X_test, y_train, y_test = download_tfds_imdb_as_text()

In [ ]:
len(X_train_new)

# 5/31 start here

In [ ]:
from dataset import download_tfds_imdb_as_text
from collections import Counter
from gensim.models import Word2Vec, KeyedVectors
import pickle
import spacy
nlp = spacy.load("en_core_web_sm", disable=["tagger", "parser"])


%load_ext autoreload
%autoreload
from wvtf2 import MyWord2Vec


In [3]:
# X_train, X_test, y_train, y_test = download_tfds_imdb_as_text()
# corpus = list()

# for s in nlp.pipe(X_train, disable=["tagger", "parser"], n_threads=4):
#     for tok in s:
#         if tok.is_digit or tok.is_punct or tok.like_url or tok.like_num or tok.like_email:
#             continue
#         corpus.append(tok.orth_.lower())

# pickle.dump(corpus, open("corpus-2020-05-31.pkl", "wb"))
    

In [4]:
corpus =  pickle.load( open("corpus-2020-05-31.pkl", "rb"))

NameError: name 'pickle' is not defined

In [ ]:
corpus[:10]

In [ ]:
embedding_size = 300
max_vocabulary_size = 100000
min_occurrence = 1
skip_window = 10
batch_size = 1000
epoch = 50


myWV = MyWord2Vec(corpus, embedding_size, max_vocabulary_size, min_occurrence, skip_window, batch_size, epoch, num_skips=2, num_sampled=5)



In [ ]:
emb, idx = myWV.train()

wv = dict()
for i, vocab in idx.items():
    wv[vocab] = emb[i,:]
    
pickle.dump(wv, open("wv_20200531-remove-punct-digit.pkl", "wb"))

In [ ]:
def foo(**kwargs):
    if "bar" in kwargs:
        print(kwargs.get("s"))
    foo1(12, **kwargs)
        
def foo1(x, bar=None, **kwargs):
    print(bar)

In [ ]:
foo(bar=5, s=44)

In [ ]:
def analyzer(x):
    bigram = list()
    for i in range(len(x)-1):
         bigram.append((x[i], x[i+1]))
    return x + bigram

In [ ]:
analyzer(["cat", "dog", "tiger"])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(analyzer=lambda x: x, binary=True, min_df=0.999999)

In [ ]:
vectorizer.fit_transform([
    ["cat", "dog", "tiger"],
    ["cat", "cat", "tiger"], 
    ["cat", "tiger", "tiger"]]
).toarray()

In [1]:
from lib.nlp_utils import spacy_tokenizer, hash_sents

In [2]:
from lib.dataset import download_tfds_imdb_as_text, download_tfds_imdb_as_text_tiny

In [3]:
dataset  = download_tfds_imdb_as_text() # tuple of (X_train, X_test, y_train, y_test)

In [4]:
X_train, X_test, y_train, y_test = dataset

In [5]:
import time
now = time.time()
X_train_tok = spacy_tokenizer(X_train)
print(time.time() - now)

70.50378704071045


In [6]:
import time
now = time.time()
X_train_tok = spacy_tokenizer(X_train, use_cache=False)
print(time.time() - now)

326.81897473335266
